In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/train/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

validation_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/test/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [4]:
def efficonvres(projection_dim):

    def apply(inputs):
        x_input = inputs

        x = layers.Conv2D(filters=projection_dim,kernel_size=3,padding="same",groups=projection_dim)(x_input)

        x1 = layers.DepthwiseConv2D(3, padding="same", use_bias=False)(x)
        x1 = layers.BatchNormalization(3)(x1)
        x1 = layers.Activation("swish")(x1)

        x1 = layers.DepthwiseConv2D(3, padding="same", use_bias=False)(x1)
        x1 = layers.LayerNormalization(epsilon=1e-6)(x1)
        x1 = layers.Activation("gelu")(x1)

        x = layers.LayerNormalization(epsilon=1e-6)(x)
        x = layers.Dense(4 * projection_dim)(x)
        x = layers.Activation("gelu")(x)
        x = layers.Dense(projection_dim)(x)

        x = tf.keras.layers.Add()([x, x1])
        return x
    return apply

dims = [32, 64, 128, 256]
inputs = tf.keras.Input(shape=(128, 128, 3))
x = layers.Conv2D(filters=128,kernel_size=3,padding="same", activation='gelu')(inputs)
for i in range(4):
    x = layers.Conv2D(filters=dims[i],kernel_size=3,padding="same", activation='gelu')(x)
    x = efficonvres(dims[i])(x)
x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
output = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=output)

In [5]:
METRICS = [
        'accuracy',
    ]
    
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
        loss='binary_crossentropy',
        metrics=METRICS
    )

In [6]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
)

Epoch 1/10
652/652 [==============================] - 314s 445ms/step - loss: 0.5156 - accuracy: 0.7349 - val_loss: 0.6407 - val_accuracy: 0.6250
Epoch 2/10
652/652 [==============================] - 288s 441ms/step - loss: 0.4389 - accuracy: 0.7506 - val_loss: 0.5263 - val_accuracy: 0.6619
Epoch 3/10
652/652 [==============================] - 288s 442ms/step - loss: 0.3874 - accuracy: 0.7889 - val_loss: 0.4752 - val_accuracy: 0.7564
Epoch 4/10
652/652 [==============================] - 288s 441ms/step - loss: 0.3508 - accuracy: 0.8363 - val_loss: 0.4439 - val_accuracy: 0.7853
Epoch 5/10
652/652 [==============================] - 288s 441ms/step - loss: 0.3230 - accuracy: 0.8660 - val_loss: 0.5641 - val_accuracy: 0.6875
Epoch 6/10
652/652 [==============================] - 288s 441ms/step - loss: 0.3035 - accuracy: 0.8752 - val_loss: 0.5430 - val_accuracy: 0.7099
Epoch 7/10
652/652 [==============================] - 288s 441ms/step - loss: 0.2888 - accuracy: 0.8829 - val_loss: 0.4803 -

In [7]:
model.save("efficonvres_Pneumonia.h5")

In [8]:
test_generator = test_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/test/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

model.evaluate(test_generator)

Found 624 images belonging to 2 classes.
78/78 [==============================] - 11s 140ms/step - loss: 0.4154 - accuracy: 0.8045


[0.4153989255428314, 0.8044871687889099]